<a href="https://colab.research.google.com/github/pachterlab/LSCHWCP_2023/blob/main/Notebooks/Figure_2/Figure_2a/create_ref_taxonomic_tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Code to create the necessary for this [iTOL tree](https://itol.embl.de/tree/731141191163851728428307):

In [1]:
import pandas as pd

Download virus ID to taxonomy mapping:

In [2]:
!wget https://raw.githubusercontent.com/pachterlab/LSCHWCP_2023/main/PalmDB/ID_to_taxonomy_mapping.csv

--2024-10-09 01:32:58--  https://raw.githubusercontent.com/pachterlab/LSCHWCP_2023/main/PalmDB/ID_to_taxonomy_mapping.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19705497 (19M) [text/plain]
Saving to: ‘ID_to_taxonomy_mapping.csv’

ID_to_taxonomy_mapp 100%[===================>]  18.79M  83.0MB/s    in 0.2s    

2024-10-09 01:32:59 (83.0 MB/s) - ‘ID_to_taxonomy_mapping.csv’ saved [19705497/19705497]



In [3]:
palmdb = pd.read_csv("ID_to_taxonomy_mapping.csv")
# Only keep columns of interest here
palmdb = palmdb[["ID", "phylum", "class", "order", "family", "genus", "species"]]
palmdb

,ID,phylum,class,order,family,genus,species
0,u1,Pisuviricota,Pisoniviricetes,Nidovirales,Coronaviridae,Betacoronavirus,Severe acute respiratory syndrome-related coro...
1,u10066,Pisuviricota,Pisoniviricetes,Nidovirales,Coronaviridae,Betacoronavirus,Severe acute respiratory syndrome-related coro...
2,u10121,Pisuviricota,Pisoniviricetes,Nidovirales,Coronaviridae,Betacoronavirus,Severe acute respiratory syndrome-related coro...
3,u10124,Pisuviricota,Pisoniviricetes,Nidovirales,Coronaviridae,Betacoronavirus,Severe acute respiratory syndrome-related coro...
4,u102609,Pisuviricota,Pisoniviricetes,Nidovirales,Coronaviridae,Betacoronavirus,Severe acute respiratory syndrome-related coro...
...,...,...,...,...,...,...,...
296556,u296608,.,.,.,.,.,.
296557,u296609,.,.,.,.,.,.
296558,u296613,.,.,.,.,.,.
296559,u296616,.,.,.,.,.,.


Add phylum "Unclassified" to all labels without classification and "Unclassified sp." labels with only species classified:

In [4]:
# ## OLD Original code prior to journal reviews:
# # Only keep records with "." across all columns as "Unclassified" or those with only a species label as "Unclassified sp."
# for index, row in palmdb.iterrows():
#     if row["phylum"] == "." and row["class"] == "." and row["order"] == "." and row["family"] == "." and row["genus"] == ".":
#         if row["species"] == ".":
#             row["phylum"] = "Unclassified"
#         else:
#             row["phylum"] = "Unclassified sp."



## NEW Updated code based on Nature Biotechnology reviews:

# Replace "." with "[unknown_label] [last known taxonomic rank] [first letter of current rank]"
# label to retain all branches in the Newick tree

unknown_label = "Unclass."

def remove_rank_letter(old_string, letter_to_remove):
    pattern = f" {letter_to_remove}"

    # Only remove the pattern if it appears at the end of the string
    if old_string.endswith(pattern):
        return old_string[: -len(pattern)]
    # Return the string unchanged if the pattern is not at the end
    return old_string

def replace_unclassified(row, current_rank, higher_rank):
    return row[current_rank].replace(
        ".",
        f"{unknown_label} {row[higher_rank]}"
        if f"{unknown_label} " not in row[higher_rank]
        else f"{remove_rank_letter(row[higher_rank], higher_rank[0].capitalize())} {current_rank[0].capitalize()}",
    )

palmdb["phylum"] = palmdb["phylum"].str.replace(".", f"{unknown_label} P", regex=False)
palmdb["class"] = palmdb.apply(lambda row: replace_unclassified(row, "class", "phylum"), axis=1)
palmdb["order"] = palmdb.apply(lambda row: replace_unclassified(row, "order", "class"), axis=1)
palmdb["family"] = palmdb.apply(lambda row: replace_unclassified(row, "family", "order"), axis=1)
palmdb["genus"] = palmdb.apply(lambda row: replace_unclassified(row, "genus", "family"), axis=1)
palmdb["species"] = palmdb.apply(lambda row: replace_unclassified(row, "species", "genus"), axis=1)

palmdb

,ID,phylum,class,order,family,genus,species
0,u1,Pisuviricota,Pisoniviricetes,Nidovirales,Coronaviridae,Betacoronavirus,Severe acute respiratory syndrome-related coro...
1,u10066,Pisuviricota,Pisoniviricetes,Nidovirales,Coronaviridae,Betacoronavirus,Severe acute respiratory syndrome-related coro...
2,u10121,Pisuviricota,Pisoniviricetes,Nidovirales,Coronaviridae,Betacoronavirus,Severe acute respiratory syndrome-related coro...
3,u10124,Pisuviricota,Pisoniviricetes,Nidovirales,Coronaviridae,Betacoronavirus,Severe acute respiratory syndrome-related coro...
4,u102609,Pisuviricota,Pisoniviricetes,Nidovirales,Coronaviridae,Betacoronavirus,Severe acute respiratory syndrome-related coro...
...,...,...,...,...,...,...,...
296556,u296608,Unclass. P,Unclass. C,Unclass. O,Unclass. F,Unclass. G,Unclass. S
296557,u296609,Unclass. P,Unclass. C,Unclass. O,Unclass. F,Unclass. G,Unclass. S
296558,u296613,Unclass. P,Unclass. C,Unclass. O,Unclass. F,Unclass. G,Unclass. S
296559,u296616,Unclass. P,Unclass. C,Unclass. O,Unclass. F,Unclass. G,Unclass. S


In [5]:
palmdb[["phylum", "class", "order", "family", "genus"]].drop_duplicates()

,phylum,class,order,family,genus
0,Pisuviricota,Pisoniviricetes,Nidovirales,Coronaviridae,Betacoronavirus
409,Negarnaviricota,Monjiviricetes,Mononegavirales,Filoviridae,Ebolavirus
475,Kitrinoviricota,Flasuviricetes,Amarillovirales,Flaviviridae,Flavivirus
590,Kitrinoviricota,Alsuviricetes,Hepelivirales,Hepeviridae,Orthohepevirus
909,Negarnaviricota,Monjiviricetes,Mononegavirales,Paramyxoviridae,Morbillivirus
...,...,...,...,...,...
277768,Pisuviricota,Pisoniviricetes,Picornavirales,Secoviridae,Sequivirus
280680,Pisuviricota,Pisoniviricetes,Picornavirales,Picornaviridae,Potamipivirus
282832,Kitrinoviricota,Magsaviricetes,Nodamuvirales,Sinhaliviridae,Unclass. Sinhaliviridae
285582,Nucleocytoviricota,Unclass. Nucleocytoviricota,Unclass. Nucleocytoviricota O,Unclass. Nucleocytoviricota F,Unclass. Nucleocytoviricota G


Make barplot showing number of viruses per phylum, class, order etc. and then overlay with how many of these can also be found in NCBI data.  

In [6]:
def df_to_dict(df):
    """
    Convert data frame to node_to_children dictionary as a first step to converting it to Newick format.

    Every row in the data frame (df) should stand for one complete branch of the tree from the root to the leaves in order
    from left to right.

    Returns formatted dictionary.

    Source:
    https://stackoverflow.com/questions/65017040/how-to-convert-pandas-dataframe-to-dictionary-for-newick-format
    """
    node_to_children = {}

    # Setting arbritrary branch length 1
    branch_length = 1

    # Iterate over dataframe row-wise, assuming that every row stands for one complete branch of the tree
    for row in df.itertuples():
        # Remove index at position 0 and elements that contain no child (".")
        row_list = [element for element in row[1:] if element != "."]

        for i in range(len(row_list)-1):
            if row_list[i] in node_to_children.keys():
                # Check if parent entry already existing
                if row_list[i+1] in node_to_children[row_list[i]].keys():
                    # If already exists --> next
                    continue
                else:
                    # If entry does not exist --> update dict and add the connection
                    node_to_children[row_list[i]].update({row_list[i+1]:branch_length})
            else:
                # Add the branching point
                node_to_children[row_list[i]] = {row_list[i+1]:branch_length}

    return node_to_children


def newickify(node_to_children, root_node) -> str:
    """
    Function to convert a dictionary to Newick format.

    Source:
    https://stackoverflow.com/questions/50003007/how-to-convert-python-dictionary-to-newick-form-format
    """
    # Setting arbritrary branch length 1
    branch_length = 1

    visited_nodes = set()

    def newick_render_node(name, distance: float) -> str:
        assert name not in visited_nodes, "Error: The tree may not be circular!"

        if name not in node_to_children:
            # Leafs
            return F'{name}:{distance}'
        else:
            # Nodes
            visited_nodes.add(name)
            children = node_to_children[name]
            children_strings = [newick_render_node(child, children[child]) for child in children.keys()]
            children_strings = ",".join(children_strings)
            return F'({children_strings}){name}:{distance}'

    newick_string = newick_render_node(root_node, branch_length) + ';'

    # Ensure no entries in the dictionary were left unused
    assert visited_nodes == set(node_to_children.keys()), "Error: some nodes aren't in the tree"

    return newick_string

Remove virus ID and species columns to create a clean tree:

In [7]:
# Remove non-relevent columns and change order of columns
phylogeny_data_clean = palmdb[["phylum", "class", "order", "family", "genus"]]

# Add "root" node
phylogeny_data_clean.insert(0, "root", "root")

## Convert data frame to node_to_children dictionary
node_to_children = df_to_dict(phylogeny_data_clean)

## Convert dictionary to Newick format
nw_string = newickify(node_to_children, root_node='root')

In [8]:
with open("palmdb_newick.txt", "w") as file:
    file.write(nw_string)

Use palmdb_newick.txt to create the [iTOL](https://itol.embl.de/) tree. Download the tree nodes from iTOL (this was done using the [iTOL annotation editor](https://itoleditor.letunic.com/) in Google Sheets):

In [9]:
!wget https://raw.githubusercontent.com/pachterlab/LSCHWCP_2023/main/Notebooks/Figure_2/Figure_2a/tree_endnodes.csv

--2024-10-09 01:33:39--  https://raw.githubusercontent.com/pachterlab/LSCHWCP_2023/main/Notebooks/Figure_2/Figure_2a/tree_endnodes.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15268 (15K) [text/plain]
Saving to: ‘tree_endnodes.csv’

tree_endnodes.csv   100%[===================>]  14.91K  --.-KB/s    in 0.001s  

2024-10-09 01:33:39 (10.8 MB/s) - ‘tree_endnodes.csv’ saved [15268/15268]



In [10]:
gtn = pd.read_csv("tree_endnodes.csv", sep=",", header=0)

# Drop unnecessary columns
gtn = gtn[["Tree node ID", "Tree node label"]]

gtn

,Tree node ID,Tree node label
0,Mimivirus,Mimivirus
1,Unclass. Nucleocytoviricota G,Unclass. Nucleocytoviricota G
2,Muromegalovirus,Muromegalovirus
3,Varicellovirus,Varicellovirus
4,Badnavirus,Badnavirus
...,...,...
425,Hunnivirus,Hunnivirus
426,Gallivirus,Gallivirus
427,Passerivirus,Passerivirus
428,Bopivirus,Bopivirus


Add species counts in PalmDB database:

In [11]:
palmdb_sp_counts = []

for end_node in gtn["Tree node ID"].values:
    palmdb_sp_counts.append(len(palmdb[palmdb.isin([end_node]).any(axis=1)]))

gtn["palmdb_sp_count"] = palmdb_sp_counts
gtn

,Tree node ID,Tree node label,palmdb_sp_count
0,Mimivirus,Mimivirus,1
1,Unclass. Nucleocytoviricota G,Unclass. Nucleocytoviricota G,1
2,Muromegalovirus,Muromegalovirus,1
3,Varicellovirus,Varicellovirus,1
4,Badnavirus,Badnavirus,1
...,...,...,...
425,Hunnivirus,Hunnivirus,41
426,Gallivirus,Gallivirus,9
427,Passerivirus,Passerivirus,21
428,Bopivirus,Bopivirus,6


Species counts in NCBI/RefSeq (for the iTOL tree shown above and in the manuscript, the NCBI species counts were retrieved from http://www.virusite.org/index.php on 2024/10/08 (previous version: 2023/03/28)):

In [12]:
import requests
import time
from tqdm import tqdm

In [13]:
ncbi_sp_counts = []
for end_node in tqdm(gtn["Tree node ID"].values, desc="Processing nodes", unit="node"):
    # NEW: Set NCBI counts to "NA" for unclassified sOTUs
    if unknown_label in end_node:
        ncbi_sp_counts.append("NA")

    else:
        time.sleep(1)
        # Note rows was set to 15000 to overwrite maximum of 25 results
        r = requests.get(f"http://www.virusite.org/index.php?nav=browse&query1={end_node}&field1=virus.name&expand=false&search_nav=virus&sort=name&order=asc&rows=15000&page=1")

        # NEW: Handle virusite web scraping restrictions
        if "Permission denied" in r.text:
            sleep = 20
            print(f"Permission denied for node {end_node}. Trying again in {sleep}s.")
            time.sleep(sleep)
            r = requests.get(f"http://www.virusite.org/index.php?nav=browse&query1={end_node}&field1=virus.name&expand=false&search_nav=virus&sort=name&order=asc&rows=15000&page=1")

        if r.status_code != 200:
            print(f"Returned status code {r.status_code} for node {end_node}.")

        df_list = pd.read_html(r.content)
        df = df_list[-1]

        ncbi_sp_counts.append(len(df))

gtn["ncbi_sp_count"] = ncbi_sp_counts
gtn

Processing nodes:  11%|█         | 48/430 [00:43<04:43,  1.35node/s]

Permission denied for node Fijivirus. Trying again in 20s.


Processing nodes:  12%|█▏        | 53/430 [01:13<16:25,  2.61s/node]

Permission denied for node Rotavirus. Trying again in 20s.


Processing nodes:  14%|█▎        | 59/430 [01:46<18:09,  2.94s/node]

...

Permission denied for node Teschovirus. Trying again in 20s.


Processing nodes: 100%|█████████▉| 429/430 [21:10<00:03,  3.66s/node]

Permission denied for node Potamipivirus. Trying again in 20s.


Processing nodes: 100%|██████████| 430/430 [21:32<00:00,  3.01s/node]


,Tree node ID,Tree node label,palmdb_sp_count,ncbi_sp_count
0,Mimivirus,Mimivirus,1,5
1,Unclass. Nucleocytoviricota G,Unclass. Nucleocytoviricota G,1,NA
2,Muromegalovirus,Muromegalovirus,1,4
3,Varicellovirus,Varicellovirus,1,22
4,Badnavirus,Badnavirus,1,77
...,...,...,...,...
425,Hunnivirus,Hunnivirus,41,2
426,Gallivirus,Gallivirus,9,3
427,Passerivirus,Passerivirus,21,2
428,Bopivirus,Bopivirus,6,1


In [14]:
# # OLD: Manually check the numbers of the "Unclassified" (0) and "Unclassified sp." (see below) endnotes for NCBI:
# # Number of Unclassified sp. for NCBI
# r = requests.get(f"http://www.virusite.org/index.php?nav=browse&query1=unclassified+viruses&field1=virus.name&expand=false&search_nav=virus&sort=name&order=asc&rows=15000&page=1")
# df_list = pd.read_html(r.content)
# df = df_list[-1]

# gtn.loc[gtn.index[gtn["Tree node ID"] == "Unclassified sp."][0], "ncbi_sp_count"] = len(df)

# # Manually change "Unclassified" to 0
# gtn.loc[gtn.index[gtn["Tree node ID"] == "Unclassified"][0], "ncbi_sp_count"] = 0

Log the counts since otherwise the NCBI counts are not visible in the plot since the PalmDB counts are much larger:

In [15]:
import numpy as np
import numpy.ma as ma

In [16]:
# Take log of values while masking 0s to keep them
gtn["palmdb_sp_count_log"] = ma.log(gtn["palmdb_sp_count"].values).filled(0)
gtn["ncbi_sp_count_log"] = ma.log(gtn["ncbi_sp_count"].replace("NA", 0).values).filled(0)

gtn

<ipython-input-16-b29937226964>:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  gtn["ncbi_sp_count_log"] = ma.log(gtn["ncbi_sp_count"].replace("NA", 0).values).filled(0)


,Tree node ID,Tree node label,palmdb_sp_count,ncbi_sp_count,palmdb_sp_count_log,ncbi_sp_count_log
0,Mimivirus,Mimivirus,1,5,0.000000,1.609438
1,Unclass. Nucleocytoviricota G,Unclass. Nucleocytoviricota G,1,NA,0.000000,0.000000
2,Muromegalovirus,Muromegalovirus,1,4,0.000000,1.386294
3,Varicellovirus,Varicellovirus,1,22,0.000000,3.091042
4,Badnavirus,Badnavirus,1,77,0.000000,4.343805
...,...,...,...,...,...,...
425,Hunnivirus,Hunnivirus,41,2,3.713572,0.693147
426,Gallivirus,Gallivirus,9,3,2.197225,1.098612
427,Passerivirus,Passerivirus,21,2,3.044522,0.693147
428,Bopivirus,Bopivirus,6,1,1.791759,0.000000


In [17]:
# Substract the NCBI counts from the PalmDB since the bars will be on top of each other in the iTOL tree
# The PalmDB bar will be "behind" the NCBI bar (unfortunately, plotting two bars next to each other is not possible in iTOL)
gtn["palmdb_sp_count_log-ncbi_sp_count_log"] = gtn["palmdb_sp_count_log"] - gtn["ncbi_sp_count_log"]

# Replace all negative values in the "palmdb_sp_count_log-ncbi_sp_count_log" column with 0
gtn.loc[gtn['palmdb_sp_count_log-ncbi_sp_count_log'] < 0, 'palmdb_sp_count_log-ncbi_sp_count_log'] = 0

In [18]:
gtn.to_csv("tree_endnodes_with_counts.csv")

On the iTOL website (using the "Datasets" tab):
1. Add the "ncbi_sp_count_log" and "palmdb_sp_count_log-ncbi_sp_count_log" to the iTOL tree as a multi value bar plot
2. Add the counts in the "palmdb_sp_count" column as text labels below the bar plot
3. Add the counts in the "ncbi_sp_count" column as text labels below the bar plot

___